### Imports

In [1]:
import main_var
mv = main_var.main_var()

from article_scraping_lib import *
from article_parsing_lib import *
from openai_module_lib import *
from embedding_keyword_module_lib import *
from visualization_module_lib import *
from dimension_reduc_lib import *
from aggregation_module_lib import *
from utils_art import *

_ENTRY_NUMBER = 1000000
_STEP_PCT = 0.01

IMPORT : main_var 
IMPORT : utils_art
IMPORT : text_analysis

IMPORT : embedding_keyword_module_lib
IMPORT : article_scraping_lib
IMPORT : article_parsing_lib
IMPORT : openai_module_lib
IMPORT : dimension_reduc 


## Querring, Scraping and saving articles
#### Querring of articles

In [ ]:
# df_query = openDFcsv(mv.query_path,mv.query_filename)
df_query = loop_scraping(number_topics=9,
                   startDate='2015-01-01',
                   endDate='2024-01-01',
                   sampling_1=10,
                   sampling_2=12,
                   save_steps=True,
                   save_final=True,
                   display=True)

#### Display Statistics

In [ ]:
plotDFstatisticsQuerry(df_query,70,False)
displayStats(df_query) 

#### Filter articles with a source that is too/not enought scrapped and source/category/date

In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
df_query_filter = filterQuerryDataset(thd_high=100,thd_low=3,display_stats=True,display_end_stats=False,save=True)
plotDFstatisticsQuerry(df_query_filter)
displayStats(df_query_filter)

In [ ]:
# categroy_l = ["WORLD","NATION","BUSINESS","TECHNOLOGY"]
# source_l = ["Newsweek","Yahoo Finance","ESPN","Fox News","World Health Organization (WHO)","Pew Research Center","The New York Times","Nature.com","Tax Foundation","Union of Concerned Scientists"]
# df3 = selectOnDf(df2,date_start="2010-01-01", date_end="2024-01-01", categroy_list=categroy_l, source_list=source_l)
# displayStats(df3)
# saveDFcsv(df3,mv.query_path,"query_fileG_cap")

### Scrap articles from the web based on the query file

In [ ]:
# df_scrap = openDFcsv(mv.scarp_path,mv.scarp_filename) #26860
df_scrap = readArticleFileTable(index_from=0,
                          index_to=_ENTRY_NUMBER,
                          save_articles=False,
                          save_final=True,
                          save_steps=True,
                          display_df=True,
                          step_pct=_STEP_PCT,
                          add_nlp=2,
                          filtered_input_df=True)

#### *Join the two results*

In [ ]:
# df_join1 = openDFcsv(mv.join1_path,mv.join1_filename)
df_join1 = joinQuerryAndParse(save=True,
                        remove_invalid=True,
                        display=True,
                        filtered_input_df=True)

## Generation of Embedding and Keyword data
#### Embedding with OpenAI API

In [2]:
# df_embdedding = openDFcsv(mv.embdedding_path,mv.embdedding_filename)
df_embdedding = mainGeneration(articleTRUEquestionFALSE=True,
                     completionTRUEembeddingFALSE=False,
                     dimension=100,
                     max_prompt=_ENTRY_NUMBER,
                     token_max_emb=7500,
                     cara_max_emb=10000,
                     save_final=True,
                     display_df_var=True,
                     save_steps=True,
                     step_pct=_STEP_PCT)

In folder :  C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/1_3_article_main/.main/  found  23074  article files.
 - #0 -  {'valid': 'VALID'} - 1099 - 5265 - 00005e0fc3f50f63d24e7aa5b99fff57fba18d6e
 - #1 -  {'valid': 'VALID'} - 1825 - 9220 - 0001da11c3d4f43777e677dea08376cfa870bba3
 - #2 -  {'valid': 'VALID'} - 826 - 3887 - 00021f030820cf263275a55d72d102d2c5927578
 - #3 -  {'valid': 'VALID'} - 1227 - 5422 - 0003a504fadc4a2ce966fa0769fa92c00fdef9db
 - #4 -  {'valid': 'VALID'} - 50 - 231 - 0003f1d6280e44ab96117f2f4c4254182ecaca83
 - #5 -  {'valid': 'VALID'} - 1583 - 7017 - 0005f299c50f22366b33c8229c9f157eef697ac6
 - #6 -  {'valid': 'VALID'} - 1139 - 5598 - 00077c89f7890b0aa4c70be1a72ca4eaef66a290
 - #7 -  {'valid': 'VALID'} - 672 - 3516 - 0009aaa91ed4d80f676ee0f0716dccc1da961079
 - #8 -  {'valid': 'VALID'} - 398 - 1436 - 0009ed484f9c3fb727047b895fd735d1e88561b6
 - #9 -  {'valid': 'VALID'} - 1379 - 5800 - 000ab5db739952f2ffc5543d7807343de8d2f599
 - #10 -  {'valid': 'VALID'} - 2550

 - #95 -  {'valid': 'VALID'} - 786 - 4464 - 00f0007243bad040145f76764b44818f841d3680
 - #96 -  {'valid': 'VALID'} - 1199 - 5701 - 00f0a90fc2443e8144934dfb3fd0ce85f533d53c
 - #97 -  {'valid': 'VALID'} - 970 - 4574 - 00f56b19950725833e3f9adbf725bf1ab0f9bec5
 - #98 -  {'valid': 'VALID'} - 3147 - 11626 - 00f8e7d5ba3f2f80e664586df05dedd9e2d4998e
 - #99 -  {'valid': 'VALID'} - 353 - 1780 - 00fb9307d899c841234b5c2a6ecb0031abf21c4a
 - #100 -  {'valid': 'VALID'} - 669 - 3489 - 00fe101cb3a674650d75a8caaa5f60e8fc2d3dbf
 - #101 -  {'valid': 'VALID'} - 1081 - 4775 - 01016004ec38a8fe72ca491af07d3ecfd1fe43b2
 - #102 -  {'valid': 'VALID'} - 625 - 3021 - 0101e3bc7c9a819e6e4fc1949abbff85950eccb9
 - #103 -  {'valid': 'VALID'} - 89 - 377 - 010321e044aa5d40f9d5ed9fb87934b1152e4fb2
 - #104 -  {'valid': 'VALID'} - 1029 - 4937 - 010562472bbddada7956262ba61289e24210c2ac
 - #105 -  {'valid': 'VALID'} - 726 - 4000 - 0107cde3c4135a7d26dbfdf0a4e2f012b5070add
 - #106 -  {'valid': 'VALID'} - 729 - 3074 - 010b517dde8

 - #191 -  {'valid': 'VALID'} - 1960 - 9283 - 020dd5bf4789b7171f7a2c44b8b254404e647e60
 - #192 -  {'valid': 'VALID'} - 802 - 4050 - 0210b14f719dda5cf05acf2bd29cdac2de048fed
 - #193 -  {'valid': 'VALID'} - 640 - 3407 - 02114903d807f9b9fbc1178a422a194ba978a5b8
 - #194 -  {'valid': 'VALID'} - 1252 - 5686 - 021524dd029c70e096e3ab631bdae8b6fa3ac7c0
 - #195 -  {'valid': 'VALID'} - 481 - 2331 - 0219792b2de124327e3d19b907e23e027ad17d17
 - #196 -  {'valid': 'VALID'} - 859 - 4656 - 021b82189f793adbc9248f80e00c24309851f633
 - #197 -  {'valid': 'VALID'} - 618 - 3284 - 021cc90dbe8863bcaa8b04845533966a15ac4d7e
 - #198 -  {'valid': 'VALID'} - 1021 - 5452 - 021cdfa0a128795953aaaf0fe53c3fd1b496c2b5
 - #199 -  {'valid': 'VALID'} - 334 - 1691 - 02216cd5055c4dfb934036ef306f78022b54f66a
 - #200 -  {'valid': 'VALID'} - 342 - 1754 - 0226642cd4b772b6908fbaf12bb14a432f0a2331
 - #201 -  {'valid': 'VALID'} - 1562 - 7861 - 02276cbc4f34c12e3fb6d91fe0df59404ff199f6
 - #202 -  {'valid': 'VALID'} - 648 - 2854 - 0229d

 - #286 -  {'valid': 'VALID'} - 263 - 1140 - 03396af438594cc30f9809504848db951ee54ec4
 - #287 -  {'valid': 'VALID'} - 762 - 3787 - 033de293495323e789771a267bfaaca737999b1c
 - #288 -  {'valid': 'VALID'} - 1224 - 6474 - 034176cd7753d7fb45428825d9dd2efd5df0db12
 - #289 -  {'valid': 'VALID'} - 708 - 3236 - 03444597a1bb676fdfa4cb0d59ee30b2972b4013
 - #290 -  {'valid': 'VALID'} - 953 - 4188 - 034764ef5c1e9592fc303b7f1c550c5699e3dc13
 - #291 -  {'valid': 'VALID'} - 793 - 3551 - 034907ac4414d5481182d48779e17683b7ae444f
 - #292 -  {'valid': 'VALID'} - 600 - 2665 - 034ec7dd9ba8ab8f01ffdbca1cbc6998ac956332
 - #293 -  {'valid': 'VALID'} - 568 - 3124 - 034ef6c4963f249e714866e963150cefcd4bdad2
 - #294 -  {'valid': 'VALID'} - 835 - 4154 - 0350a3c7506147a485400ac67f738b3574d7f1dc
 - #295 -  {'valid': 'VALID'} - 469 - 2188 - 03584c95b9c625f6c865ac63024463760d0abe95
 - #296 -  {'valid': 'VALID'} - 112 - 598 - 0359ae1bd2113f55886bbde9c6d37ab4b01ec125
 - #297 -  {'valid': 'VALID'} - 927 - 4581 - 035a2a06c

 - #382 -  {'valid': 'VALID'} - 180 - 785 - 045f1a4161a9775418297248972aa3fe100ebb03
 - #383 -  {'valid': 'VALID'} - 558 - 2610 - 046426c890ff11dfd29b6877a1c3a431529b008d
 - #384 -  {'valid': 'VALID'} - 611 - 3264 - 046447eba4e173dd8dac64c63a2cb7a9e9b0c83c
 - #385 -  {'valid': 'VALID'} - 1527 - 7491 - 04694d83b92a083cd068a3bae9c3c98270ac5a93
 - #386 -  {'valid': 'VALID'} - 857 - 4804 - 046bb5d44b2b4e7c68833cf0507f05b197a9147e
 - #387 -  {'valid': 'VALID'} - 85 - 401 - 046d7cfc5aecb0fc30469545b1b69a4b0d3a57b6
 - #388 -  {'valid': 'VALID'} - 338 - 1639 - 046db063f2a952ba3d3e584a67296334389a5d36
 - #389 -  {'valid': 'VALID'} - 481 - 2313 - 046dd9feffd54050dcedc5e9ee26add9eaef3011
 - #390 -  {'valid': 'VALID'} - 526 - 2482 - 047233768c81afb0acd63ceccfbf53c90f9fa8b6
 - #391 -  {'valid': 'VALID'} - 763 - 3842 - 04725bc42631c14549594de5fb6b58db308530cc
 - #392 -  {'valid': 'VALID'} - 1389 - 6605 - 0473dae59d97fa3ae297f0a7ec00812bb9799682
 - #393 -  {'valid': 'VALID'} - 400 - 1962 - 0476d44f39

...

 - #22729 -  {'valid': 'VALID'} - 1462 - 8061 - fc23644d9e0cc42f73862006a9835c8b9d3d722d
 - #22730 -  {'valid': 'VALID'} - 43 - 248 - fc240263aff11b4c4e2155eb9b6153d74e4d6708
 - #22731 -  {'valid': 'VALID'} - 1092 - 5390 - fc295ad5c9eaae352aae40884b0e4dfd57cf0cb5
 - #22732 -  {'valid': 'VALID'} - 18 - 80 - fc29a31441ff0ab275d22fdca9fe49212a309129
 - #22733 -  {'valid': 'VALID'} - 1014 - 5526 - fc2acb455b7ec79b0f191f3d6cc714a3f03fe635
 - #22734 -  {'valid': 'VALID'} - 1086 - 5527 - fc2bc825f98e8801b7daf753e5b3bdd9b10436f3
 - #22735 -  {'valid': 'VALID'} - 561 - 2901 - fc35f026769318474e9b7cb087e95012987db3d7
 - #22736 -  {'valid': 'VALID'} - 3653 - 15887 - fc3919d8ecd603b4ff732f41702994165ea6b97e
 - #22737 -  {'valid': 'VALID'} - 2167 - 12301 - fc3daf8f87038d5dafa8863b8b16d2d764c46633
 - #22738 -  {'valid': 'VALID'} - 1306 - 5660 - fc3e3a2a5484cca794564eee813a365c3384ccbf
 - #22739 -  {'valid': 'VALID'} - 320 - 1408 - fc4076013df947bf99ec683cbbbb85ad668b1faa
 - #22740 -  {'valid': 'VALI

 - #22822 -  {'valid': 'VALID'} - 175 - 765 - fd0c48bb7d96b06f5e806e22113df002ffd8c7ed
 - #22823 -  {'valid': 'VALID'} - 54 - 262 - fd0f0ebf18ded6f4c645ce067aa3fc27453d81ac
 - #22824 -  {'valid': 'VALID'} - 823 - 4111 - fd11bfaa33fd23fc8e0b36dd4a7abd8e50e87527
 - #22825 -  {'valid': 'VALID'} - 1074 - 5310 - fd14a323d67d53c19c7b9c8cac7be52a404c5a6c
 - #22826 -  {'valid': 'VALID'} - 1472 - 6177 - fd150d8ebf17c4a07669f1c9da0f70e93d803de9
 - #22827 -  {'valid': 'VALID'} - 786 - 4303 - fd175fb3e043e97ffbbaa6d2e693e9ab024335a9
 - #22828 -  {'valid': 'VALID'} - 891 - 3685 - fd18114f3a2ab228901bdd17b2b8950c451246b1
 - #22829 -  {'valid': 'VALID'} - 304 - 1670 - fd1a71c7646427131c194901191a9f140918f62c
 - #22830 -  {'valid': 'VALID'} - 1251 - 6013 - fd1b851748486405b80db9f4512bb561c107a5e6
 - #22831 -  {'valid': 'VALID'} - 329 - 1735 - fd1f76e4f97d9fdd2a7ad41bfa2a622990f92bca
 - #22832 -  {'valid': 'VALID'} - 368 - 1216 - fd22cc19033080e959384c8d567cb4dbb6ca07b3
 - #22833 -  {'valid': 'VALID'} 

 - #22915 -  {'valid': 'VALID'} - 115 - 547 - fe2f9321e9269030b2a3cb2abf488c74b787830b
 - #22916 -  {'valid': 'VALID'} - 409 - 2051 - fe30860a771154e40b1531efa036ee8c6d7fc313
 - #22917 -  {'valid': 'VALID'} - 115 - 597 - fe3212251c8fae7a8f64856bce690169fcea6b13
 - #22918 -  {'valid': 'VALID'} - 1063 - 6015 - fe352bb41fe29b81171474b864c5321967594c78
 - #22919 -  {'valid': 'VALID'} - 1794 - 8171 - fe3997af0a265b756b3e78625d3d893fa1ecea67
 - #22920 -  {'valid': 'VALID'} - 1078 - 4037 - fe39b77ee17e10b167d932b81db02a957214f56b
 - #22921 -  {'valid': 'VALID'} - 816 - 4049 - fe3c369d70244a34033c60761fc099fa4a8a6ceb
 - #22922 -  {'valid': 'VALID'} - 771 - 4193 - fe3faf4fbb76c96493b2be7249a312413df458bc
 - #22923 -  {'valid': 'VALID'} - 137 - 747 - fe426e1012127cc5f1ebdd5683e3af3aaebd0c6b
 - #22924 -  {'valid': 'VALID'} - 1207 - 5932 - fe440c2ec58ede4f2c0c3c5682e7c25b2f1b1117
 - #22925 -  {'valid': 'VALID'} - 1110 - 5467 - fe475e3db195f7598340f261532ccf7c73e37709
 - #22926 -  {'valid': 'VALID'

 - #23008 -  {'valid': 'VALID'} - 1045 - 5024 - ff2f7ad43ddc0cb34070cb1305c1d452d2c12929
 - #23009 -  {'valid': 'VALID'} - 531 - 2557 - ff3a3094c75a1ad78c084543876e633071af9f1f
 - #23010 -  {'valid': 'VALID'} - 750 - 3600 - ff3a8d7bfd9c5fe80c1d87f7b8a4b1791e4ff270
 - #23011 -  {'valid': 'VALID'} - 3923 - 18402 - ff3e3b2865d9bc42c13bb3fcbe38f3a5635f4249
 - #23012 -  {'valid': 'VALID'} - 214 - 956 - ff4d85d44104df8a41f9fc95393035528d3763ff
 - #23013 -  {'valid': 'VALID'} - 103 - 564 - ff4e128605539755182cd218bacfdec4e71bbaf0
 - #23014 -  {'valid': 'VALID'} - 1486 - 7329 - ff4ff03684ac940073cf3530211f8d019f1a163e
 - #23015 -  {'valid': 'VALID'} - 594 - 2779 - ff535d362b280cc020926df85a2255053c4cc66e
 - #23016 -  {'valid': 'VALID'} - 867 - 4096 - ff53a80b7399cbba397aa776abb49441c50270f5
 - #23017 -  {'valid': 'VALID'} - 519 - 2591 - ff5792aa4032c924cf2ef8d862c36eb2022f118b
 - #23018 -  {'valid': 'VALID'} - 346 - 1527 - ff5c115128ab83aa9c20d119a5964c8fcd3af1ce
 - #23019 -  {'valid': 'VALID'

,hash_key,i_model,i_dimensions,i_encoding_format,i_user,o_data,o_token_input,o_token_total,valid
0,00005e0fc3f50f63d24e7aa5b99fff57fba18d6e,text-embedding-3-small,100,float,name_test,"[0.17722228, 0.015102669, 0.056136545, 0.21115...",1099,1099,VALID
1,0001da11c3d4f43777e677dea08376cfa870bba3,text-embedding-3-small,100,float,name_test,"[0.116140224, -0.042566903, 0.07320826, 0.1081...",1825,1825,VALID
2,00021f030820cf263275a55d72d102d2c5927578,text-embedding-3-small,100,float,name_test,"[0.13898131, 0.14415655, -0.011063003, 0.19050...",826,826,VALID


##### Extract embedding from file

In [3]:
emb_mat = extractEmbeddingFromFile(_ENTRY_NUMBER,True)

C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/2_1_embdedding_main/.main/embdedding_file
Input dataframe shape : (23074, 9)
df_main :                                         hash_key                 i_model  i_dimensions i_encoding_format     i_user                                             o_data  o_token_input  o_token_total  valid
0      00005e0fc3f50f63d24e7aa5b99fff57fba18d6e  text-embedding-3-small           100             float  name_test  [0.17722228, 0.015102669, 0.056136545, 0.21115...           1099           1099  VALID
1      0001da11c3d4f43777e677dea08376cfa870bba3  text-embedding-3-small           100             float  name_test  [0.116140224, -0.042566903, 0.07320826, 0.1081...           1825           1825  VALID
2      00021f030820cf263275a55d72d102d2c5927578  text-embedding-3-small           100             float  name_test  [0.13898131, 0.14415655, -0.011063003, 0.19050...            826            826  VALID
3      0003a504fadc4a2ce966fa0769fa92c00fdef9d

,hash_key,i_model,i_dimensions,i_encoding_format,i_user,o_data,o_token_input,o_token_total,valid
0,00005e0fc3f50f63d24e7aa5b99fff57fba18d6e,text-embedding-3-small,100,float,name_test,"[0.17722228, 0.015102669, 0.056136545, 0.21115...",1099,1099,VALID
1,0001da11c3d4f43777e677dea08376cfa870bba3,text-embedding-3-small,100,float,name_test,"[0.116140224, -0.042566903, 0.07320826, 0.1081...",1825,1825,VALID
2,00021f030820cf263275a55d72d102d2c5927578,text-embedding-3-small,100,float,name_test,"[0.13898131, 0.14415655, -0.011063003, 0.19050...",826,826,VALID
3,0003a504fadc4a2ce966fa0769fa92c00fdef9db,text-embedding-3-small,100,float,name_test,"[0.034137163, 0.190266, -0.028268237, 0.102514...",1227,1227,VALID
4,0003f1d6280e44ab96117f2f4c4254182ecaca83,text-embedding-3-small,100,float,name_test,"[0.016711636, -0.082255684, -0.15815344, -0.09...",50,50,VALID
...,...,...,...,...,...,...,...,...,...
23069,fff564c55bebbc8114799cc6a3f310b0322e9094,text-embedding-3-small,100,float,name_test,"[0.13610439, -0.0036302656, -0.14663129, -0.00...",1342,1342,VALID
23070,fff8e229ad3449b85b3b4479eda7b64c83700f44,text-embedding-3-small,100,float,name_test,"[-0.0423533, 0.034358982, 0.10479593, 0.232953...",1027,1027,VALID
23071,fff988b32618b761a8e8358559dddb655d06520f,text-embedding-3-small,100,float,name_test,"[0.016940664, -0.021444917, -0.11371772, 0.066...",978,978,VALID
23072,ffff52bdb39e1fa008807af05bec26b439c16a8e,text-embedding-3-small,100,float,name_test,"[-0.06786892, 0.10157983, 0.014239955, 0.16766...",31,31,VALID


emb_mat :                                                    1               2              3               4              5              6               7              8              9              10   ...            91             92             93             94            95             96             97             98             99             100
hash_key                                                                                                                                                                                         ...                                                                                                                                                     
00005e0fc3f50f63d24e7aa5b99fff57fba18d6e   0.17722228     0.015102669    0.056136545      0.21115096   -0.026217617    0.030535812    -0.059837855     0.01599495   -0.073012754    0.008807137  ...    -0.16100702     0.04282945   -0.051377717    0.030557843   0.012679192    -0.05212679    0.111832

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
hash_key,,,,,,,,,,,,,,,,,,,,,
00005e0fc3f50f63d24e7aa5b99fff57fba18d6e,0.17722228,0.015102669,0.056136545,0.21115096,-0.026217617,0.030535812,-0.059837855,0.01599495,-0.073012754,0.008807137,...,-0.16100702,0.04282945,-0.051377717,0.030557843,0.012679192,-0.05212679,0.111832455,0.02337554,0.06825393,-0.04609013
0001da11c3d4f43777e677dea08376cfa870bba3,0.116140224,-0.042566903,0.07320826,0.10815741,0.17250669,-0.007258764,-0.07491191,0.0030772171,-0.1249992,-0.012089828,...,0.014955609,-0.054857515,-0.07437647,0.1456377,0.10747595,-0.049211137,-0.09954181,-0.032831762,0.08503645,0.113219686
00021f030820cf263275a55d72d102d2c5927578,0.13898131,0.14415655,-0.011063003,0.19050865,0.20940952,-0.076765984,0.0027094982,-0.041964408,0.13035592,0.017766433,...,-0.094579294,0.09615437,-0.019219624,-0.028951315,-0.10102959,-0.036639165,0.09285422,-0.06229033,0.13793127,-0.065477975
0003a504fadc4a2ce966fa0769fa92c00fdef9db,0.034137163,0.190266,-0.028268237,0.102514006,0.05082132,0.08006344,-0.03470099,0.09277517,-0.042312656,-0.09267266,...,0.034495965,0.06191846,0.08862336,0.022642782,0.13162798,0.07596288,-0.17037828,0.031343658,0.19272633,-0.09959236
0003f1d6280e44ab96117f2f4c4254182ecaca83,0.016711636,-0.082255684,-0.15815344,-0.09289638,0.031436417,-0.112411685,-0.0092830155,0.082079075,0.09395604,0.07452904,...,-0.0737343,0.027330257,-0.032319464,-0.09263147,0.09404434,0.15462126,0.14534928,-0.08720074,0.16239206,0.07324862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff564c55bebbc8114799cc6a3f310b0322e9094,0.13610439,-0.0036302656,-0.14663129,-0.0061484533,0.0012991227,-0.057711616,0.07149906,0.13517281,-0.11737956,0.040500607,...,-0.16200243,0.08570571,0.12026747,0.07853252,0.14076231,-0.08333017,-0.12762699,0.053659227,-0.01907418,-0.016815089
fff8e229ad3449b85b3b4479eda7b64c83700f44,-0.0423533,0.034358982,0.10479593,0.23295349,0.18308222,-0.03365482,-0.12674923,0.32209218,-0.016537461,0.08015025,...,0.015802233,-0.06875939,-0.08193137,-0.05119675,0.028415024,0.010277664,0.03684426,0.01130802,0.0140521815,-0.053060707
fff988b32618b761a8e8358559dddb655d06520f,0.016940664,-0.021444917,-0.11371772,0.0669203,0.07843247,-0.012518309,-0.1429193,0.14413604,-0.16528848,0.12279642,...,-0.08390777,-0.013313865,-0.16893867,0.026697926,-0.03589362,-0.16266781,0.09621549,0.16575645,0.014284912,-0.055829316


##### Use dime,tion reduction on emneded data

In [4]:
df_embdedding2 = generateDimReducedDF(n_components=3,
                                          norm_output=True,
                                      active_sel=[True,True,True,False])

Input dataframe shape : (23074, 8)


#### Keyword extraction and NLP analysis

In [5]:
# df = openDFcsv(mv.keyword_path,mv.keyword_filename)
df_keyword = mainKeywordWF(entry_limit=_ENTRY_NUMBER,
              common_word_max=500,
              add_nlp_stats=True,
              nlp_source_col="word_combined_all")

,count
keyword,
eisenberg,1
desanctis,1
thruline,1
reuben,1
gloria,1
...,...
to,814
in,843
s,968


Unique keywords : 9302   Sum of occurences : 53936   (5.8 avg)
Unique keywords : 490   Sum of occurences : 20883   (42.62 avg)


,count
keyword,
lifting,1
prevalence,1
wasting,1
peeters,1
hosting,1
...,...
business,459
science,500
health,504


Unique keywords : 14395   Sum of occurences : 62039   (4.31 avg)
Unique keywords : 520   Sum of occurences : 25832   (49.68 avg)
0_f                object
1_f                object
2_f                object
3_f                object
4_f                object
5_f                object
6_f                object
7_f                object
8_f                object
9_f                object
10_f               object
11_f               object
12_f               object
13_f               object
14_f               object
15                 object
16                 object
17                 object
18                 object
19                 object
20                 object
21                 object
22                 object
23                 object
24                 object
25                 object
word_count_f        int64
word_combined_f    object
0_s                object
1_s                object
2_s                object
3_s                object
4_s                object
5_s          

 - Generate NLP for article #159  (char:193)
 - Generate NLP for article #160  (char:201)
 - Generate NLP for article #161  (char:148)
 - Generate NLP for article #162  (char:156)
 - Generate NLP for article #163  (char:84)
 - Generate NLP for article #164  (char:188)
 - Generate NLP for article #165  (char:198)
 - Generate NLP for article #166  (char:156)
 - Generate NLP for article #167  (char:154)
 - Generate NLP for article #168  (char:193)
 - Generate NLP for article #169  (char:141)
 - Generate NLP for article #170  (char:172)
 - Generate NLP for article #171  (char:194)
 - Generate NLP for article #172  (char:149)
 - Generate NLP for article #173  (char:183)
 - Generate NLP for article #174  (char:158)
 - Generate NLP for article #175  (char:144)
 - Generate NLP for article #176  (char:205)
 - Generate NLP for article #177  (char:201)
 - Generate NLP for article #178  (char:193)
 - Generate NLP for article #179  (char:201)
 - Generate NLP for article #180  (char:173)
 - Generate

...

,title_quer,keywords_list,word_count_f_t,word_combined_f_t,word_count_f_k,word_combined_f_k,word_count_s_t,word_combined_s_t,word_count_s_k,word_combined_s_k,word_combined_all,word_count_all,word_combined_all_sel,word_count_all_sel
hash_key,,,,,,,,,,,,,,
42dba98da775f90b8c020fd8a20c29b43e7e3023,‘Top Chef’ Boston Episode 11 Recap: ‘Sous Your...,"['chef', 'elimination', 'melissa', 'really', '...",9,top chef boston episode recap sous your daddy ...,15,chef elimination melissa really doesnt challen...,5,boston your episode recap chef,8,boston really going challenge episode recap do...,top chef boston episode recap sous your daddy ...,24,boston your episode recap chef boston really ...,13
479039fb0ea67ef91620420785286eb670a0e7f6,A few minutes with ... a guy who likes the top...,"['im', 'detroit', 'maybe', 'little', 'driving'...",14,a few minutes with a guy who likes the top dow...,13,im detroit maybe little driving theres weather...,5,who press down free with,7,press free think im little theres day,a few minutes with a guy who likes the top dow...,27,who press down free with press free think im ...,12
0cac25fa3dc0b19e36198475bae1d4910dedc763,Top Chef Boston Episode 10: Foie Nah - Eater,"['noodles', 'college', 'chef', 'nah', 'foie', ...",7,top chef boston episode foie nah eater,14,noodles college chef nah foie ramen challenge ...,3,boston chef episode,5,boston chef challenge episode college,top chef boston episode foie nah eater noodle...,21,boston chef episode boston chef challenge epi...,8


Generated DF shape : (4551, 17)


,tb.pol,tb.sub,tb.polaj,tb.pos,tb.neg,vs.pos,vs.neu,vs.neg,vs.comp,ts.neg,ts.neu,ts.pos,tb.class,vs.class,ts.class,al.pos,al.neg
hash_key,,,,,,,,,,,,,,,,,
42dba98da775f90b8c020fd8a20c29b43e7e3023,0.350000,0.350000,0.675000,0.142467,0.857533,0.071,0.870,0.059,0.0771,0.397526,0.551467,0.051007,False,True,False,0.333333,0.666667
479039fb0ea67ef91620420785286eb670a0e7f6,0.126157,0.498148,0.563079,0.031196,0.968804,0.327,0.673,0.000,0.8807,0.007177,0.597077,0.395746,False,True,True,0.666667,0.333333
0cac25fa3dc0b19e36198475bae1d4910dedc763,0.500000,0.500000,0.750000,0.770518,0.229482,0.135,0.742,0.122,0.0772,0.029139,0.893224,0.077638,True,True,True,1.000000,0.000000


Output DF shape : (4551, 31)


,title_quer,keywords_list,word_count_f_t,word_combined_f_t,word_count_f_k,word_combined_f_k,word_count_s_t,word_combined_s_t,word_count_s_k,word_combined_s_k,...,vs.neg,vs.comp,ts.neg,ts.neu,ts.pos,tb.class,vs.class,ts.class,al.pos,al.neg
hash_key,,,,,,,,,,,,,,,,,,,,,
42dba98da775f90b8c020fd8a20c29b43e7e3023,‘Top Chef’ Boston Episode 11 Recap: ‘Sous Your...,"['chef', 'elimination', 'melissa', 'really', '...",9,top chef boston episode recap sous your daddy ...,15,chef elimination melissa really doesnt challen...,5,boston your episode recap chef,8,boston really going challenge episode recap do...,...,0.059,0.0771,0.397526,0.551467,0.051007,False,True,False,0.333333,0.666667
479039fb0ea67ef91620420785286eb670a0e7f6,A few minutes with ... a guy who likes the top...,"['im', 'detroit', 'maybe', 'little', 'driving'...",14,a few minutes with a guy who likes the top dow...,13,im detroit maybe little driving theres weather...,5,who press down free with,7,press free think im little theres day,...,0.000,0.8807,0.007177,0.597077,0.395746,False,True,True,0.666667,0.333333
0cac25fa3dc0b19e36198475bae1d4910dedc763,Top Chef Boston Episode 10: Foie Nah - Eater,"['noodles', 'college', 'chef', 'nah', 'foie', ...",7,top chef boston episode foie nah eater,14,noodles college chef nah foie ramen challenge ...,3,boston chef episode,5,boston chef challenge episode college,...,0.122,0.0772,0.029139,0.893224,0.077638,True,True,True,1.000000,0.000000


#### *Joining Quering/Scraping/Embedding/Keywords*

In [6]:
# df_join2 = openDFcsv(mv.join2_path,mv.join2_filename)
joinAllDF()

C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/3_1_join_main/.main/ join2_filetest


,title_quer,link,published,source_url,source_title,category,year,year_month,pk,url_list,...,vs.neg_k,vs.comp_k,ts.neg_k,ts.neu_k,ts.pos_k,tb.class,vs.class,ts.class,al.pos_k,al.neg_k
hash_key,,,,,,,,,,,,,,,,,,,,,
42dba98da775f90b8c020fd8a20c29b43e7e3023,‘Top Chef’ Boston Episode 11 Recap: ‘Sous Your...,https://news.google.com/rss/articles/CBMiTmh0d...,2015-01-15,https://observer.com,Observer,TOP,2015,2015-01,CBMiTmh0dHBzOi8vb2JzZXJ2ZXIuY29tLzIwMTUvMDEvdG...,['observer'],...,0.059,0.0771,0.397526,0.551467,0.051007,False,True,False,0.333333,0.666667
479039fb0ea67ef91620420785286eb670a0e7f6,A few minutes with ... a guy who likes the top...,https://news.google.com/rss/articles/CBMiVGh0d...,2015-01-11,https://www.freep.com,Detroit Free Press,TOP,2015,2015-01,CBMiVGh0dHBzOi8vd3d3LmZyZWVwLmNvbS9zdG9yeS9uZX...,['freep'],...,0.000,0.8807,0.007177,0.597077,0.395746,False,True,True,0.666667,0.333333
0cac25fa3dc0b19e36198475bae1d4910dedc763,Top Chef Boston Episode 10: Foie Nah - Eater,https://news.google.com/rss/articles/CBMiQWh0d...,2015-01-08,https://www.eater.com,Eater,TOP,2015,2015-01,CBMiQWh0dHBzOi8vd3d3LmVhdGVyLmNvbS8yMDE1LzEvOC...,['eater'],...,0.122,0.0772,0.029139,0.893223,0.077638,True,True,True,1.000000,0.000000
fcee664fa9bb6a73de43636b766fcad766e04677,Top Chef recap: 'The Final Battle of Beantown'...,https://news.google.com/rss/articles/CBMiM2h0d...,2015-01-22,https://ew.com,Entertainment Weekly News,TOP,2015,2015-01,CBMiM2h0dHBzOi8vZXcuY29tL3JlY2FwL3RvcC1jaGVmLX...,['ew'],...,0.078,0.7896,0.005248,0.418656,0.576096,True,True,True,1.000000,0.000000
31e4e714f0fd541a29c03b477452a0589c2d4f77,‘Top Chef’ Boston Episode 10 Recap: ‘For Julia...,https://news.google.com/rss/articles/CBMiX2h0d...,2015-01-08,https://www.boston.com,Boston.com,TOP,2015,2015-01,CBMiX2h0dHBzOi8vd3d3LmJvc3Rvbi5jb20vZm9vZC9mb2...,['boston'],...,0.000,0.2732,0.012336,0.913599,0.074066,True,True,True,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ee4c73dbc96b30e45d4c85ba0aef4af34ac64a57,NSA's Top Brazilian Political and Financial Ta...,https://news.google.com/rss/articles/CBMiYGh0d...,2015-07-04,https://theintercept.com,The Intercept,TOP,2015,2015-07,CBMiYGh0dHBzOi8vdGhlaW50ZXJjZXB0LmNvbS8yMDE1Lz...,['theintercept'],...,0.000,0.5994,0.043357,0.912366,0.044278,False,True,True,0.666667,0.333333
5926355b21f233dfc9af281f890f5b340ec80e38,Top Gear: Jeremy Clarkson's swansong draws 5.3...,https://news.google.com/rss/articles/CBMiUGh0d...,2015-06-29,https://www.theguardian.com,The Guardian,TOP,2015,2015-06,CBMiUGh0dHBzOi8vd3d3LnRoZWd1YXJkaWFuLmNvbS9tZW...,['theguardian'],...,0.156,-0.3818,0.235337,0.622183,0.142480,True,False,False,0.333333,0.666667
90a30e8d360f152eaf47d8ddd2421b3a52eb162a,Conway ranks top five Bears players - ChicagoB...,https://news.google.com/rss/articles/CBMiTmh0d...,2015-06-29,https://www.chicagobears.com,ChicagoBears.com,TOP,2015,2015-06,CBMiTmh0dHBzOi8vd3d3LmNoaWNhZ29iZWFycy5jb20vbm...,['chicagobears'],...,0.000,0.7184,0.009264,0.818379,0.172356,True,True,True,1.000000,0.000000


In [ ]:
df_gen = generateDimReducedDF(3,True,[True,True,True,True])

In [ ]:
# df_join1 = openDFcsv(mv.join1_path,mv.join1_filename)
# df_embdedding = openDFcsv(mv.embdedding_path,mv.embdedding_filename)
# df = openDFcsv(mv.keyword_path,mv.keyword_filename)

# df = openDFcsv(mv.join2_path,mv.join2_filename)



In [ ]:
df = openDFcsv(mv.join2_path,mv.join2_filename)
val1 = len(list(df.keys()))
print(val1)
rename_dict = {"hash_key":"Primary Key", "title_quer":"Main Title","link":"Google Link","published":"Date Publication","source_url":"Real Link","source_title":"Name of the source","category":"Main Category","year":"Year","year_month":"Year & Month", "url_TLD":"url_extention","keywords_list":"List Of Keywords","tb.sent":"Number of sentences","tb.noun":"Number of Nouns","tb.word":"Number of Words","tb.char":"Number of Character","tb.pol":"Polarity","tb.sub":"Subjectivity","tb.polaj":"Polatity beetween 1-0","tb.pos":"Positivity_1","tb.neg":"Negativity_1","vs.pos":"Positivity_2","vs.neg":"Negativity_2","ts.pos":"Positivity_3","ts.neg":"Negativity_3","al.pos":"Positivity_all","al.neg":"Negativity_all"}
val2 = len(list(rename_dict.keys()))
print(val2)
# print(len(list(rename_dict.keys())))
# print(len(list(rename_dict.values())))
df = openDFcsv(mv.join2_path,mv.join2_filename)
df.rename(columns=rename_dict, inplace=True)
val3 = len(list(df.keys()))
print(val3)

In [ ]:
col_list = ["category","source_title","year_month","url_TLD","year"]
df_list_len = calculateStatsColList(df,col_list,"len",display_df=True)
df_list_npl = calculateStatsColList(df,col_list,"nlp",display_df=True,display_stats=False) 

In [ ]:
# df = openDFcsv(mv.join1_path,mv.join1_filename)
# genrateKeywordExtract(df, 9641,500,"keywords_list")
#df = mainKhttp://localhost:8888/notebooks/OneDrive/Desktop/Article_LLM/main_codebase/MAIN_CODE.ipynb#eywordWF(9641,1000)

In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
# df3 = selectOnDf(df2,"","")
# plotDFstatisticsQuerry(df,70)
# displayStats(df)
# saveDFcsv(df3,mv.query_path,mv.query_filename)



# df = loop_scraping(9,'1999-03-27','2024-03-19',sampling_1=5,sampling_2=5,save_steps=True,save_final=True,display=True)
# plotDFstatisticsQuerry(df,70)
# displayStats(df)

# df = openDFcsv(mv.query_path,mv.query_filename)
# df2 =  filterQuerryDataset(df,150,10,True,True,True)
# plotDFstatisticsQuerry(df2,70,True)
# displayStats(df2)

# df = openDFcsv(mv.query_path,mv.query_filename)
df_querry = readArticleFileTable(index_from=0,index_to=200,save_articles=False,save_final=True,save_steps=True,display_df=True,step_pct=0.01,add_nlp=2,filtered_input_df=True)
# df_join = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)# df_embd = mainGeneration(True,False,100,99999999,7500,9999,True,True,True,0.005)
# emb_mat = extractEmbeddingFromFile(9999999999)
# df_key = mainKeywordWF(99999999999999,1000)
# df_final = mainJoinOut()

In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
# df3 = selectOnDf(df2,"","")
# plotDFstatisticsQuerry(df,70)
# displayStats(df)
# saveDFcsv(df3,mv.query_path,mv.query_filename)


max_num = 10000000
step_pct=0.005
df = loop_scraping(9,'2000-01-01','2024-01-01',sampling_1=5,sampling_2=5,save_steps=True,save_final=True,display=True)
df2 =  filterQuerryDataset(df,150,10,True,True,True)
df_querry = readArticleFileTable(index_from=0,index_to=max_num,save_articles=True,save_final=True,save_steps=True,display_df=True,step_pct=step_pct,add_nlp=2,filtered_input_df=True)
df_join = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)
df_embd = mainGeneration(True,False,10,max_num,7500,1000,True,True,True,step_pct)
emb_mat = extractEmbeddingFromFile(max_num)
df_key = mainKeywordWF(max_num,100)
df_final = mainJoinOut()

In [ ]:
df = loop_scraping(9,'2000-01-01','2024-01-01',sampling_1=1,sampling_2=1,save_steps=True,save_final=True,display=True)

In [ ]:
df = openDFcsv(mv.join1_path,mv.join1_filename)
# col_list = ["category"] #,"url_TLD" #,"source_title","year_month",
# col_list = ["published"]
col_list = ["category","year","source_title","year_month"]#
df_list1 = calculateStatsColList(df,col_list,"len",display_df=True)
df_list2 = calculateStatsColList(df,col_list,"nlp",display_df=True,display_stats=True) #,out_raw=False

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(col_list)) :
    col_agg = col_list[i]
    table = df_list2[i]
    table.plot(kind='bar')
    plot = df_list[2].plot.barh(y='polarity_per_count',use_index=True, legend=True, fontsize= 8, title="test") #, figsize=(5, 5) #,ax=axes[0],ay=axes[0], ylabel=""


In [ ]:
# fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
# df_list[0].ix[5].plot(kind='bar'); plt.axhline(0, color='k')
    

In [ ]:
import plotly.express as px

fig = px.line(df_list2[1], x=df_list2[1].index, y="count") #, color='country'
fig.show()
fig = px.line(df_list2[1], x=df_list2[1].index, y="polarity_per_count") #, color='country'
fig.show()
fig = px.line(df_list2[1], x=df_list2[1].index, y="positivity_per_count") #, color='country'
fig.show()
fig = px.line(df_list2[1], x=df_list2[1].index, y="negativity_per_count") #, color='country'
fig.show()

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Life expentancy progression of countries per continents'),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=["Asia", "Europe", "Africa","Americas","Oceania"],
        value=["Americas", "Oceania"],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(continents):
    df = px.data.gapminder() # replace with your own data source
    mask = df.continent.isin(continents)
    fig = px.line(df[mask], 
        x="year", y="lifeExp", color='country')
    return fig


app.run_server(debug=True)

In [ ]:
d1 = {"d1":1}
d2 = {"d2":2}
d3 = d1|d2
print(d3)